# Lab5. Validation
### Cross Validation
+ The Set of Train, Valid, Test 
+ k-Fold with Stratify
+ Cross Validation Score

## Parameter Tuning
+ Grid Search
+ Random Search

## Ensemble
+ Voting Ensemble
+ Stacking, Average Blending

In [ ]:
import os
from os.path import join
import copy
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import sklearn

import matplotlib.pyplot as plt

adult_path = join('sample_data', 'adult_data.csv')
column_path = join('sample_data', 'adult_names.txt')

adult_columns = list()
for l in open(column_path):
    adult_columns = l.split()

In [ ]:
data = pd.read_csv(adult_path, names = adult_columns)
label = data['income']

del data['income']
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

### Pandas get_dummies 함수를 사용해 범주형 변수를 One-Hot Encoding하고, label data를 0,1 로 변경한다. 

In [ ]:
data.shape

In [ ]:
data = pd.get_dummies(data)
label = label.map(lambda x : 0 if x =='>50K' else 1)

In [ ]:
data.shape

In [ ]:
label.sum()

## Cross Validation
### 1. Train, Valid, Test Set
- 훈련, 검증, 테스트 데이터라고 부르는 3가지를 설명한다.
* Train Data : 모델을 학습하는데 사용하는 데이터 (모델이 알고 있는 학습할 데이터)
* Valid Data : 학습한 모델의 성능을 검증하는 데이터 (모델이 모르는 학습하지 않을 데이터, 모델 검증에 사용하는 데이터)
* Test Data : 학습한 모델로 예측할 데이터 (모델이 모르는 예측할 데이터)

<img src='./img/train_val_test.png' style='height : 500px' >


- Machine Learning에서 Validation 데이터가 왜 필요한지에 대한 부분은 참조 링크를 남겨두었으니 확인하시면 좋겠다.

In [ ]:
print('ones : {:.2f}%'.format((np.sum(label==1, axis=0)/len(data))*100))
print('zeros : {:.2f}%'.format((np.sum(label==0, axis=0)/len(data))*100))

In [ ]:
from sklearn.model_selection import train_test_split

# (Train, Valid), Test 분할
x, x_test, y, y_test = train_test_split(data, label, test_size=0.2, stratify=label, shuffle=True)

In [ ]:
# Train, Valid 분할
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, stratify=y, shuffle=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(random_state=2019)
# Train 데이터로 학습
lr.fit(x_train, y_train)

In [ ]:
# Valid 데이터로 검증
y_pred_val = lr.predict(x_valid)
print('로지스틱 회귀 검증 데이터 정확도 :  {:.2f}%'.format(accuracy_score(y_valid, y_pred_val)*100))

In [ ]:
# Test 데이터로 모델 평가
y_pred = lr.predict(x_test)
print('로지스틱 회귀 테스트 데이터 정확도 : {:.2f}%'.format(accuracy_score(y_test, y_pred)*100))

### 2. k-fold with stratify
- k-fold는 data를 k개로 쪼개는 것을 말한다.
- 일반적으로 Cross Validation에서 사용되며, Dataset을 k개로 쪼개어 k-1개로 모델을 학습하고, 1개로 모델을 검증한다.
- k개로 데이터를 쪼개면, 모든 fold에 대해(하나의 fold를 선택하여) 검증하는 방식으로 k번 다른 Dataset으로 학습한 모델을 검증할 수 있다.

![kfold](./img/kfold.png)

#### Stratify, 계층적 k-fold는 무엇인가?
- k-fold는 데이터의 정렬 유무와 분류할 클래스의 비율에 상관없이 순서대로 데이터를 분할하는 특징이 있다.
- 하지만, 분류할 클래스의 비율이 다르다면 어떻게 될까? 
- 그런 경우에는, 각 fold가 학습 Dataset을 대표한다고 말하기 어려워진다.
- 한 fold에 특정 클래스가 많이 나올수도, 적게 나올수도 있기 때문이다. 
- Stratified k-fold는 그러한 문제점을 해결하기 위해 제안되었다.
- k개의 fold도 분할한 이후에도, 전체 훈련 데이터의 클래스 비율과 각 fold가 가지고 있는 클래스의 비율을 맞추어 준다는 점이 기존의 k-fold와의 다른 특징이다. 

##### k-fold
![kfold_example](./img/kfold_example.png)

##### Stratified k-fold
![stratified_kfold_example](./img/stratified_kfold_example.png)

- k-fold 실습을 위해 iris 데이터를 불러온다.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

kf_data = iris.data
kf_label = iris.target
kf_columns = iris.feature_names

In [ ]:
kf_data = pd.DataFrame(kf_data, columns = kf_columns)
kf_data.head()

In [ ]:
kf_label

#### k-Fold
- k-fold는 말 그대로 데이터를 k개로 나눈다.
- k의 개수를 조절하여 몇개의 fold를 만들지 결정할 수 있다.

- k-fold는 sklearn의 model_selection 패키지에 있다.

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)

In [ ]:
for i, (trn_idx, val_idx) in enumerate(kf.split(kf_data.values, kf_label)) :
    trn_data, trn_label = kf_data.values[trn_idx, :], kf_label[trn_idx]
    val_data, val_label = kf_data.values[val_idx, :], kf_label[val_idx]
    
    print('{} Fold, trn label\n {}'.format(i, trn_label))
    print('{} Fold, val label\n {}\n'.format(i, val_label))

#### stratify k-Fold

- Stratified k-fold는 sklearn의 model_selection package에 있다.

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)

In [ ]:
for i, (trn_idx, val_idx) in enumerate(skf.split(kf_data, kf_label)) :
    trn_data, trn_label = kf_data.values[trn_idx,:], kf_label[trn_idx]
    val_data, val_label = kf_data.values[val_idx,:], kf_label[val_idx]
    
    print('{} Fold, trn label\n {}'.format(i, trn_label))
    print('{} Fold, val label\n {}\n'.format(i, val_label))

#### Cross Validation 해보기
- Stratified k-fold를 이용해 Cross Validation을 진행해 보자.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

val_scores = list()

for i, (trn_idx, val_idx) in enumerate(skf.split(kf_data, kf_label)) :
    trn_data, trn_label = kf_data.values[trn_idx, :], kf_label[trn_idx]
    val_data, val_label = kf_data.values[val_idx, :], kf_label[val_idx]
    
    # 모델 정의
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=2019)
    
    # 모델 학습
    clf.fit(trn_data, trn_label)

    # 훈련, 검증 데이터 정확도 확인
    trn_acc = clf.score(trn_data, trn_label)*100
    val_acc = clf.score(val_data, val_label)*100
    print('{} Fold, train Accuracy : {:.2f}%, validation Accuracy : {:.2f}%'.format(i, trn_acc, val_acc))
    
    val_scores.append(val_acc)

# 교차 검증 정확도 평균 계산하기
print('Cross Validation Score : {:.2f}%'.format(np.mean(val_scores)))

#### Cross Validation Score
- 방금 전 반복문을 사용해 Cross Validation을 진행해 봤다.
- 그런데 Sklearn에는 한번에 k-fold Cross Validation Score를 계산하는 cross_val_score 함수를 제공한다. 
- Parameter로 cv에 숫자를 전달하면, 그 숫자 만큼의 fold를 만들어 Cross Validation(CV)을 진행하고, kfold 객체를 전달하면 해당 객체에 맞게 데이터를 분할하여 CV Score를 계산한다.
- cross_val_score 함수는 fold 개수대로 Score를 반환하며, 해당 스코어들의 평균을 계산해 모델의 성능을 가늠해볼 수 있다.

* 기본적으로 cross_val_score 함수는 입력 Label 값이 클래스로 나누어진 분류 모델인 경우 StratifiedKFold를 적용한다.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# 숫자로 전달하는 경우
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=2019)
print('랜덤 포레스트 k-Fold CV Score(Acc) : {}'.format(np.mean(cross_val_score(rf, kf_data, kf_label, cv=skf))))

#### fold 객체를 전달하는 경우
- print('Random Forest k-Fold CV Score(Acc) : {:.2f}%'.format()
- print('Random Forest Stratify k-Fold CV Score(Acc) : {:.2f}%'.format()

## Parameter Tuning
### GridSearch 
- 모델에는 여러가지 parameter가 들어간다.
- SVM의 경우 Soft, Hard 마진의 정도를 결정하는 'C' 커널 함수를 결정하는 'kernel', 특정 커널에서 얼마나 세세하게 볼것인지를 결정하는 'gamma' 등.
- Parameter를 어떻게 결정하느냐에 따라 모델이 잘 학습하거나 잘 학습하지 못하는 경우가 발생할 수 있다.
- Sklearn에서 가장 쉽게 제공하는 Parameter Tuning 함수로 GridSearchCV 라는 함수가 있다. 
- 해당 함수에 각 Parameter에 사용할 수치 list를 전달하면, 해당 함수는 parameter들의 조합을 모두 시도해보며, 가장 좋은 성능의 Parameter를 찾게 된다. 

- 간단히 GridSearchCV 함수를 사용해 Random Forest의 n_estimator, max_depth parameter 중 가장 좋은 parameter 조합을 찾아본다.
- GridSearchCV 함수는 Sklearn의 model_selection package에 있다.

#### 1) 모델 정의 및 불러오기

In [ ]:
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()

In [ ]:
params = {'n_estimators' : [50, 100, 150, 200],
          'max_depth' : [5, 10 ,15, 20],
          'min_samples_split': [2, 5, 10]}

clf = GridSearchCV(RandomForestClassifier(), params, cv=skf)

#### 2) 모델 학습하기

In [ ]:
clf.fit(kf_data, kf_label)

#### 3,4) 예측 및 결과 확인

In [ ]:
print('GridSearchCV best score : {:.2f}%, best_params : {}'.format(clf.best_score_*100, clf.best_params_))

#### Scikit-Optimize
- GridSearch의 단점은 사용자가 직접 Parameter에 들어갈 값들의 list를 지정해주어야 한다는 단점이 있다.
- Sklearn library 내에 존재하지는 않지만, Scikit-Optimize(이하, skopt)라는 library를 간단히 소개하려고 한다.
- skopt는 각 Parameter에 들어갈 값들의 최대, 최소 범위를 결정해주고 파라미터 값의 분포 Scale을 결정해주어 Parameter tuning을 자동화 시켜주는 library이다.
- 아래 참조 링크에 Skopt 링크가 있으니 확인해보면 좋겠다.

## Ensemble
- 개인적으로 Ensemble은 Machine Learning의 꽃이라고 생각한다. 
- 단일 모델로 좋은 성능을 이끄는 것도 중요하지만, 서로 다른 모델의 다양성을 고려하여 결과를 이끌어내는 Ensemble은 응용할 수 있는 방법이 매우 많다. 
- 그 중 대표적인 3가지 Ensemble에 대해 실습하고 배워보도록 한다. 

### 1. Voting Ensemble
- 이름에서 알 수 있듯이 각자의 모델이 투표를 하여 클래스를 선택하는 방식의 Emsemble이다.
- Voting Ensemble은 Sklearn 자체적으로 모델로써 지원을 하며, 사용하기도 매우 쉽다. 

- 다시 Adult Dataset으로 돌아와 Ensemble을 통해 기존 단일 모델보다 좋은 결과를 얻어보도록 하자.

- Voting Classifier는 Sklearn의 ensemble package에 있다.

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.ensemble import VotingClassifier
clfs = [('LR', LogisticRegression()), ('RF', RandomForestClassifier(max_depth=5)), ('MLP', MLPClassifier()) ]

vote_clf = VotingClassifier(clfs)

#### 2) 모델 학습하기

In [ ]:
vote_clf.fit(x_train, y_train)

In [ ]:
print('Cross Validation Acc : {:.2f}%'.format(vote_clf.score(x_valid, y_valid)*100))

#### 3) 결과 예측하기

In [ ]:
y_pred = vote_clf.predict(x_test)

#### 4) 결과 확인하기

In [ ]:
print('Voting Ensemble Acc : {:.2f}%'.format(vote_clf.score(x_test, y_test)*100))

### 2. Bagging, Average Blending
- Emsemble 기법 중 Kaggle에서 가장 많이 사용되는 기법이면서 쉬운 기법이다.
- Average Blending에서 회귀의 경우 각 모델들이 예측한 결과 값을 n으로 나누어 합친다.
- 분류의 경우에는 각 클래스에 해당하는 확률을 n으로 나누어 합치고, 그 중 가장 높은 확률 값을 갖는 클래스를 택하는 방식이다. 

In [ ]:
# 단일 모델에서의 Random Forest 성능
clf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=2019)
clf.fit(x_train, y_train)
print('Single Random Forest Acc : {:.2f}%'.format(clf.score(x_test, y_test)*100))

In [ ]:
val_scores = list()

y_pred = np.zeros_like(y_test, dtype=np.float)

for i, (trn_idx, val_idx) in enumerate(skf.split(x, y)) :
    trn_data, trn_label = x.values[trn_idx, :], y.values[trn_idx]
    val_data, val_label = x.values[val_idx, :], y.values[val_idx]
    
    # 모델 정의
    clf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=2019)
    
    # 모델 학습
    clf.fit(trn_data, trn_label)
    trn_acc = clf.score(trn_data, trn_label)*100
    val_acc = clf.score(val_data, val_label)*100
    print('{} Fold, train Accuracy : {:.2f}%, validation Accuracy : {:.2f}%'.format(i, trn_acc, val_acc))
    
    val_scores.append(val_acc)
    y_pred += (clf.predict_proba(x_test)[:, 1] / skf.n_splits)
    
# Mean Validation Score
print('Cross Validation Score : {:.2f}%'.format(np.mean(val_scores)))

In [ ]:
# 확률을 이진 라벨로 변경해줍니다.
y_pred = [0 if y < 0.5 else 1 for y in y_pred]
print('Average Blending Acc : {:.2f}%'.format(accuracy_score(y_test, y_pred)*100))

### Reference
- Validation 데이터가 필요한 이유 : https://3months.tistory.com/118
- Sklearn, KFold : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
- Sklearn, StratifedKFold : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
- Sklearn, Compare with KFold, StratifedKFold : https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
- Sklearn, Cross Validation Score : https://www.google.com/url?q=http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html&sa=U&ved=0ahUKEwiGxeHhqubhAhUKV7wKHbFhDrcQFggEMAA&client=internal-uds-cse&cx=016639176250731907682:tjtqbvtvij0&usg=AOvVaw0rIHEJ1ltDaghFv1bvPeRO
- Sklearn, GridSearchCV : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- Sklearn, Voting Classifier : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html
- Scikit-Optimize, Documentation : https://scikit-optimize.github.io 